Source of code from the data file used below is from the original authors stata code.

https://www.openicpsr.org/openicpsr/project/113484/version/V1/view?path=/openicpsr/113484/fcr:versions/V1/P2016_1118_data&type=folder

In [1]:
import pandas as pd
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from statsmodels.formula.api import ols
from linearmodels.panel import PanelOLS


file_path = '/Users/rebeccluo/Downloads/US_Paid_leave_analysis.dta'

# Load the file into a DataFrame
df = pd.read_stata(file_path)
birth_vars = [f'_IBirth_{i}' for i in range(2, 52)]  # Birth dummies from _IBirth_2 to _IBirth_51
birxpos_vars = [f'_IBirXpos_{i}_1' for i in range(2, 52)]  # Event-study dummies _IBirXpos_2_1 to _IBirXpos_50_1
llbirth_vars = [f'_LlBirth_{2}_1'] + [f'_LlBirth_{i}_1' for i in range(8, 51)]   # Reference period dummies from _LlBirth_8 to _LlBirth_50
llbipos_vars = [f'_LlBiXpos_{2}_1'] + [f'_LlBiXpos_{i}_1' for i in range(8, 51)]  # Include _LlBiXpos_2_1 and _LlBiXpos_8_1 to _LlBiXpos_50_1  # Event-study reference period _LlBiXpos_8_1 to _LlBiXpos_50_1


/Users/rebeccluo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/rebeccluo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

df = df.dropna(subset=['age_group'])
df_no_college = df[df['lt_college'] == 0]
df_no_college .set_index(['sippid', 'months'], inplace=True)
print(df_no_college.head())

                      ssuid  spanel  swave  srefmon  rhcalmn  rhcalyr  \
sippid months                                                           
1.0    29.0    019092404375    1996      9        4       12     1998   
       35.0    019092404375    1996      2        1        5     1996   
       23.0    019092404375    1996     11        3        7     1999   
       32.0    019092404375    1996      3        4       12     1996   
       13.0    019092404375    1996      3        1        9     1996   

               tfipsst  epppnum  esex     wpfinwgt  ...  _LlBiXpos_46_1  \
sippid months                                       ...                   
1.0    29.0         12      102     2  5819.273926  ...               0   
       35.0         12      102     2  4369.268066  ...               0   
       23.0         12      102     2  6447.158203  ...               0   
       32.0         12      102     2  4976.591797  ...               0   
       13.0         12      102     2 

In [3]:
import warnings

# Suppress all warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    year_dummies = pd.get_dummies(df_no_college['rhcalyr'], drop_first=True)  # Time dummy for year
    birth_dummies = pd.get_dummies(df_no_college[birth_vars], drop_first=False)  # Dummies for birth_vars
    state_dummies = pd.get_dummies(df_no_college['state'], drop_first=True)  # Dummies for states (assuming 'state' column)

    # Step 2: Create interaction terms between birth_vars and time (rhcalyr)
    birth_time_interactions = pd.DataFrame(index=df_no_college.index)
    for birth_col in birth_dummies.columns:
        for year_col in year_dummies.columns:
            interaction_name = f'{birth_col}_time_{year_col}'
            birth_time_interactions[interaction_name] = birth_dummies[birth_col] * year_dummies[year_col]

    # Step 3: Create interaction terms between birth_vars and state
    birth_state_interactions = pd.DataFrame(index=df_no_college.index)
    for birth_col in birth_dummies.columns:
        for state_col in state_dummies.columns:
            interaction_name = f'{birth_col}_state_{state_col}'
            birth_state_interactions[interaction_name] = birth_dummies[birth_col] * state_dummies[state_col]



    state_year_interactions = pd.DataFrame(index=df_no_college.index)
    for state_col in state_dummies.columns:
        for year_col in year_dummies.columns:
            interaction_name = f'{state_col}_time_{year_col}'
            state_year_interactions[interaction_name] = state_dummies[state_col] * year_dummies[year_col]

#Combine all dummies and interaction terms into a final DataFrame
X = pd.concat([year_dummies,state_year_interactions,birth_dummies, birth_time_interactions, birth_state_interactions, df_no_college[llbipos_vars]], axis=1)

# Step 5: Add constant term for the regression
X = sm.add_constant(X)

# Step 6: Define the dependent variable (y)
y = df_no_college["rm_lfp"]  # Assuming 'rm_lfp' is your dependent variable for labor force participation

In [4]:

model = PanelOLS(y, X, entity_effects=True, time_effects=True,weights=df_no_college['end_weight'],drop_absorbed=True,check_rank=False)
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results)

/var/folders/nk/v5qz7_1x3hb9nxp2rz8sq6800000gn/T/ipykernel_97661/1495542213.py:2: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

2012, _IBirth_2_time_1999, _IBirth_2_time_2002, _IBirth_2_time_2006, _IBirth_2_time_2007, _IBirth_2_time_2011, _IBirth_2_time_2012, _IBirth_3_time_1999, _IBirth_3_time_2007, _IBirth_3_time_2011, _IBirth_3_time_2012, _IBirth_4_time_1999, _IBirth_4_time_2007, _IBirth_4_time_2011, _IBirth_4_time_2012, _IBirth_5_time_1999, _IBirth_5_time_2007, _IBirth_5_time_2011, _IBirth_5_time_2012, _IBirth_6_time_1999, _IBirth_6_time_2007, _IBirth_6_time_2011, _IBirth_6_time_2012, _IBirth_7_time_1999, _IBirth_7_time_2007, _IBirth_7_time_2010, _IBirth_7_time_2011, _IBirth_7_time_2012, _IBirth_8_time_1999, _IBirth_8_time_2007, _IBirth_8_time_2011, _IBirth_8_time_2012, _IBirth_9_time_1999, _IBirth_9_time_2007, _IBirth_9_time_2011, _IBirth_9_time_2012, _IBirth_10_time_1999, _IBirth_10_time_2007, _IBirth_10_time_2011, _IBirth_10_t

                          PanelOLS Estimation Summary                           
Dep. Variable:                 rm_lfp   R-squared:                        0.0904
Estimator:                   PanelOLS   R-squared (Between):             -0.3253
No. Observations:               35586   R-squared (Within):               0.0905
Date:                Sun, Oct 20 2024   R-squared (Overall):             -0.2063
Time:                        00:58:44   Log-likelihood                    3236.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      20.443
Entities:                         950   P-value                           0.0000
Avg Obs:                       37.459   Distribution:              F(1072,33517)
Min Obs:                       2.0000                                           
Max Obs:                       48.000   F-statistic (robust):         -7.351e+08
                            

In [5]:
from scipy.stats import chi2
test_vars = ['_LlBiXpos_22_1', '_LlBiXpos_23_1', '_LlBiXpos_24_1', 
             '_LlBiXpos_25_1', '_LlBiXpos_26_1', '_LlBiXpos_27_1', 
             '_LlBiXpos_28_1']

# Step 3: Calculate the joint test
# Extract the coefficients and covariance matrix for the test variables
coefs = results.params[test_vars]
cov_matrix = results.cov.loc[test_vars, test_vars]

wald_statistic = coefs.T @ np.linalg.inv(cov_matrix) @ coefs

# Number of restrictions (the number of variables being tested)
num_restrictions = len(test_vars)

# Calculate the p-value using the chi-squared distribution
p_value = 1 - chi2.cdf(wald_statistic, df=num_restrictions)

# Output the results
print("Joint test for mth -3 to +3:")
print(f'Wald Test Statistic: {wald_statistic}')
print(f'p-value: {p_value}')

Joint test for mth -3 to +3:
Wald Test Statistic: 4.327664812364683
p-value: 0.741357489899391


In [6]:

# Now do the same for b_X6 if needed
b_X6 = pd.Series(index=range(1, 50), dtype=float)

for j in range(1, 50):
    if 1 <= j <= 7:
        b_X6[j] = 0  # Set to 0 for indices 1 to 7
    else:
        var_name = f'_LlBiXpos_{j}_1'  # Name of the variable corresponding to the coefficient
        if var_name in results.params:  # Check if this variable was in the regression model
            b_X6[j] = results.params[var_name]  # Assign the coefficient from the regression
        else:
            b_X6[j] = None  # Handle the case where the variable might not be in the results

# Create the time column with values from -24 to 24
time_column = list(range(-24, 25))

# Create a DataFrame with the time column and b_X3
b_X6_df = pd.DataFrame({
    'time': time_column,
    'b_X3': b_X6
})

# Print the DataFrame
print(b_X6_df) #creating and storing the event-study DiD estimates


    time      b_X3
1    -24  0.000000
2    -23  0.000000
3    -22  0.000000
4    -21  0.000000
5    -20  0.000000
6    -19  0.000000
7    -18  0.000000
8    -17  0.058101
9    -16 -0.004431
10   -15 -0.006704
11   -14 -0.015708
12   -13 -0.042720
13   -12 -0.014882
14   -11 -0.021646
15   -10 -0.032069
16    -9 -0.053281
17    -8 -0.066079
18    -7 -0.022122
19    -6 -0.000356
20    -5  0.008952
21    -4 -0.008341
22    -3 -0.000598
23    -2 -0.037870
24    -1 -0.027420
25     0 -0.048247
26     1 -0.064494
27     2 -0.055958
28     3 -0.096722
29     4 -0.054689
30     5 -0.051661
31     6 -0.040103
32     7 -0.027887
33     8 -0.064959
34     9 -0.063199
35    10 -0.106839
36    11 -0.102615
37    12 -0.110861
38    13 -0.075906
39    14 -0.109740
40    15 -0.083469
41    16 -0.084938
42    17 -0.065918
43    18 -0.074609
44    19 -0.086218
45    20 -0.108527
46    21 -0.118756
47    22 -0.079735
48    23 -0.050179
49    24 -0.055776


In [21]:
file_path = '/Users/rebeccluo/Downloads/US_Paid_leave_analysis.dta'

# Load the file into a DataFrame
df = pd.read_stata(file_path)

In [22]:
df = df.dropna(subset=['age_group'])
df_college = df[df['lt_college'] == 1]
df_college .set_index(['sippid', 'months'], inplace=True)
print(df_college.head())

                      ssuid  spanel  swave  srefmon  rhcalmn  rhcalyr  \
sippid months                                                           
2.0    40.0    019133038995    1996      8        2        6     1998   
       17.0    019133038995    1996     11        4        8     1999   
       30.0    019133038995    1996      3        1        9     1996   
       45.0    019133038995    1996      3        3       11     1996   
       24.0    019133038995    1996      3        2       10     1996   

               tfipsst  epppnum  esex     wpfinwgt  ...  _LlBiXpos_46_1  \
sippid months                                       ...                   
2.0    40.0         12      102     2  2807.805908  ...               0   
       17.0         12      102     2  2987.406982  ...               0   
       30.0         12      102     2  3431.020752  ...               0   
       45.0         12      102     2  3446.817139  ...               0   
       24.0         12      102     2 

In [25]:
import warnings

# Suppress all warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    year_dummies = pd.get_dummies(df_college['rhcalyr'], drop_first=True)  # Time dummy for year
    birth_dummies = pd.get_dummies(df_college[birth_vars], drop_first=False)  # Dummies for birth_vars
    state_dummies = pd.get_dummies(df_college['state'], drop_first=True)  # Dummies for states (assuming 'state' column)

    # Step 2: Create interaction terms between birth_vars and time (rhcalyr)
    birth_time_interactions = pd.DataFrame(index=df_college.index)
    for birth_col in birth_dummies.columns:
        for year_col in year_dummies.columns:
            interaction_name = f'{birth_col}_time_{year_col}'
            birth_time_interactions[interaction_name] = birth_dummies[birth_col] * year_dummies[year_col]

    # Step 3: Create interaction terms between birth_vars and state
    birth_state_interactions = pd.DataFrame(index=df_college.index)
    for birth_col in birth_dummies.columns:
        for state_col in state_dummies.columns:
            interaction_name = f'{birth_col}_state_{state_col}'
            birth_state_interactions[interaction_name] = birth_dummies[birth_col] * state_dummies[state_col]



    state_year_interactions = pd.DataFrame(index=df_college.index)
    for state_col in state_dummies.columns:
        for year_col in year_dummies.columns:
            interaction_name = f'{state_col}_time_{year_col}'
            state_year_interactions[interaction_name] = state_dummies[state_col] * year_dummies[year_col]

#Combine all dummies and interaction terms into a final DataFrame
X = pd.concat([year_dummies,state_year_interactions,birth_dummies, birth_time_interactions, birth_state_interactions, df_college[llbipos_vars]], axis=1)

# Step 5: Add constant term for the regression
X = sm.add_constant(X)

# Step 6: Define the dependent variable (y)
y = df_college["rm_lfp"]  # Assuming 'rm_lfp' is your dependent variable for labor force participation

In [26]:

model = PanelOLS(y, X, entity_effects=True, time_effects=True,weights=df_college['end_weight'],drop_absorbed=True,check_rank=False)
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results)

/var/folders/nk/v5qz7_1x3hb9nxp2rz8sq6800000gn/T/ipykernel_66780/45623200.py:2: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

2012, Florida_time_2012, Texas_time_2012, _IBirth_2_time_1999, _IBirth_2_time_2002, _IBirth_2_time_2006, _IBirth_2_time_2007, _IBirth_2_time_2011, _IBirth_2_time_2012, _IBirth_3_time_1999, _IBirth_3_time_2006, _IBirth_3_time_2007, _IBirth_3_time_2011, _IBirth_3_time_2012, _IBirth_4_time_1999, _IBirth_4_time_2007, _IBirth_4_time_2011, _IBirth_4_time_2012, _IBirth_5_time_1999, _IBirth_5_time_2007, _IBirth_5_time_2011, _IBirth_5_time_2012, _IBirth_6_time_1999, _IBirth_6_time_2007, _IBirth_6_time_2011, _IBirth_6_time_2012, _IBirth_7_time_1999, _IBirth_7_time_2007, _IBirth_7_time_2011, _IBirth_7_time_2012, _IBirth_8_time_1999, _IBirth_8_time_2007, _IBirth_8_time_2011, _IBirth_8_time_2012, _IBirth_9_time_1999, _IBirth_9_time_2007, _IBirth_9_time_2011, _IBirth_9_time_2012, _IBirth_10_time_1999, _IBirth_10_time_2007, 

                          PanelOLS Estimation Summary                           
Dep. Variable:                 rm_lfp   R-squared:                        0.0679
Estimator:                   PanelOLS   R-squared (Between):             -0.1718
No. Observations:               68022   R-squared (Within):               0.0679
Date:                Thu, Oct 17 2024   R-squared (Overall):             -0.0798
Time:                        17:59:05   Log-likelihood                -1.325e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      18.344
Entities:                        1867   P-value                           0.0000
Avg Obs:                       36.434   Distribution:              F(1068,65040)
Min Obs:                       2.0000                                           
Max Obs:                       48.000   F-statistic (robust):             361.79
                            

In [27]:
from scipy.stats import chi2
test_vars = ['_LlBiXpos_22_1', '_LlBiXpos_23_1', '_LlBiXpos_24_1', 
             '_LlBiXpos_25_1', '_LlBiXpos_26_1', '_LlBiXpos_27_1', 
             '_LlBiXpos_28_1']

# Step 3: Calculate the joint test
# Extract the coefficients and covariance matrix for the test variables
coefs = results.params[test_vars]
cov_matrix = results.cov.loc[test_vars, test_vars]

wald_statistic = coefs.T @ np.linalg.inv(cov_matrix) @ coefs

# Number of restrictions (the number of variables being tested)
num_restrictions = len(test_vars)

# Calculate the p-value using the chi-squared distribution
p_value = 1 - chi2.cdf(wald_statistic, df=num_restrictions)

# Output the results
print("Joint test for mth -3 to +3:")
print(f'Wald Test Statistic: {wald_statistic}')
print(f'p-value: {p_value}')

Joint test for mth -3 to +3:
Wald Test Statistic: 15.988119489628508
p-value: 0.025225124709235547


In [31]:

# Now do the same for b_X6 if needed
b_X6 = pd.Series(index=range(1, 50), dtype=float)

for j in range(1, 50):
    if 1 <= j <= 7:
        b_X6[j] = 0  # Set to 0 for indices 1 to 7
    else:
        var_name = f'_LlBiXpos_{j}_1'  # Name of the variable corresponding to the coefficient
        if var_name in results.params:  # Check if this variable was in the regression model
            b_X6[j] = results.params[var_name]  # Assign the coefficient from the regression
        else:
            b_X6[j] = None  # Handle the case where the variable might not be in the results

# Create the time column with values from -24 to 24
time_column = list(range(-24, 25))

# Create a DataFrame with the time column and b_X3
b_X6_df = pd.DataFrame({
    'time': time_column,
    'b_X3': b_X6
})

# Print the DataFrame
print(b_X6_df) #creating and storing the event-study DiD estimates


    time      b_X3
1    -24  0.000000
2    -23  0.000000
3    -22  0.000000
4    -21  0.000000
5    -20  0.000000
6    -19  0.000000
7    -18  0.000000
8    -17 -0.002920
9    -16  0.075823
10   -15  0.064568
11   -14  0.080537
12   -13  0.028631
13   -12  0.067607
14   -11  0.014008
15   -10  0.019978
16    -9  0.014698
17    -8 -0.001360
18    -7  0.014192
19    -6  0.047974
20    -5  0.090552
21    -4  0.088338
22    -3  0.118702
23    -2  0.130401
24    -1  0.097743
25     0  0.159728
26     1  0.132210
27     2  0.080829
28     3  0.057282
29     4  0.072921
30     5  0.055497
31     6  0.043830
32     7  0.058117
33     8  0.002976
34     9  0.014759
35    10  0.017534
36    11 -0.016126
37    12  0.015255
38    13  0.022096
39    14  0.018790
40    15  0.074461
41    16  0.037330
42    17  0.038766
43    18  0.075454
44    19  0.025124
45    20  0.055927
46    21  0.034248
47    22  0.064192
48    23  0.035889
49    24 -0.007565
